# Foresight Path Safety Testing

Test `is_path_safe()` for different body heights: ankle, torso, head. Safe vs blocked paths.

In [ ]:
import sys
from pathlib import Path
root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(root))
import numpy as np
import matplotlib.pyplot as plt
from foresight import VoxelCostmap, GROUND_LAYER, TORSO_LAYER, HEAD_LAYER

## 1. Setup costmap with test obstacles

In [ ]:
cm = VoxelCostmap()

# Table blocking torso height (0.7-0.8m)
cm.add_obstacle_box(4, 3, 0.7, 6, 5, 0.8)

# Low curb/object at ankle (0-0.2m)
cm.add_obstacle_box(3, 6, 0, 5, 7, 0.2)

# Hanging branch at head (2-2.3m)
cm.add_obstacle_box(7, 2, 2.0, 9, 4, 2.3)

print("Obstacles: table (torso), curb (ankle), branch (head)")

## 2. Define test paths

In [ ]:
paths = [
    ((1, 4, 0.5), (8, 4, 0.5), 'Path A: through table', TORSO_LAYER),
    ((1, 4, 0.5), (8, 4, 0.5), 'Path A: ankle only', GROUND_LAYER),
    ((4, 1, 0.5), (4, 8, 0.5), 'Path B: past curb', GROUND_LAYER),
    ((4, 1, 0.5), (4, 8, 0.5), 'Path B: torso (clears curb)', TORSO_LAYER),
    ((5, 1, 1.5), (10, 4, 1.5), 'Path C: under branch', HEAD_LAYER),
    ((5, 1, 1.5), (10, 4, 1.5), 'Path C: head clearance', HEAD_LAYER),
]

In [ ]:
for start, end, label, layer in paths:
    safe = cm.is_path_safe(start, end, body_height_range=layer)
    print(f"{label}: {'SAFE' if safe else 'BLOCKED'}")

## 3. Visualize paths on 2D costmap slices

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 5))

test_paths = [
    ((1, 4), (8, 4), TORSO_LAYER, 'Path A (torso) - blocked by table'),
    ((4, 1), (4, 8), GROUND_LAYER, 'Path B (ankle) - blocked by curb'),
    ((5, 1), (10, 4), HEAD_LAYER, 'Path C (head) - blocked by branch'),
]
layers = [GROUND_LAYER, TORSO_LAYER, HEAD_LAYER]
titles = ['Ground layer', 'Torso layer', 'Head layer']

for ax, layer, title in zip(axes, layers, titles):
    sl = cm.get_layer_slice(layer)
    ax.imshow(sl.T, origin='lower', cmap='RdYlGn_r', vmin=0, vmax=100,
              extent=[0, 20, 0, 20], aspect='equal')
    ax.set_title(title)
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    
    for (sx, sy), (ex, ey), _, _ in test_paths:
        ax.plot([sx, ex], [sy, ey], 'b-', alpha=0.6, lw=2)
        ax.plot(sx, sy, 'go', markersize=10)
        ax.plot(ex, ey, 'rs', markersize=10)

plt.suptitle('Path safety by height layer (green=start, red=goal)')
plt.tight_layout()
plt.show()

## 4. Interactive path checker

In [ ]:
def check_path(start_xy, end_xy, height=1.0, height_range=0.5):
    """Check if path is safe for user at given height."""
    z_min = max(0, height - height_range / 2)
    z_max = min(3, height + height_range / 2)
    start = (start_xy[0], start_xy[1], height)
    end = (end_xy[0], end_xy[1], height)
    return cm.is_path_safe(start, end, body_height_range=(z_min, z_max))

# Example: wheelchair user (torso height ~1.2m)
safe = check_path((2, 2), (8, 8), height=1.2, height_range=0.6)
print(f"Path (2,2) -> (8,8) for torso height 1.2m: {'SAFE' if safe else 'BLOCKED'}")